In [92]:
import requests
import pandas as pd
import os
import sys

In [93]:
# Senior Role String Identifiers
senior_roles = ['senior', 'sr', 'lead', 'principal', 'staff', 'director', 'head', 'chief', 'vp', 'cto', 'cfo', 'cio', 'coo', 'ceo', 'president', 'founder', 'partner', 'executive', 'expert', 'guru']

# Numbered Roles for Seniority
seniority = [ 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 3, 4, 5, 6, 7, 8, 9, 10]
seniority = [str(x) for x in seniority]

In [94]:
# site https://h1bdata.info/index.php?em=&job=[job]&city=&year=2023
def get_data(job, year):
    url = f'https://h1bdata.info/index.php?em=&job={job}&city=&year={year}'
    response = requests.get(url)
    return response.text

def process_data(data):
    data = data.split('<table border="1" id="myTable" class="tablesorter tablesorter-blue hasStickyHeaders">')[1]
    data = data.split('</table>')[0]
    data = data.split('<tr>')
    data = data[1:]
    data = [x.split('</td>') for x in data]
    data = [[x.split('<td>') for x in y] for y in data]
    data = [[x[1] for x in y if len(x) > 1] for y in data]
    data = [[x.replace('<a href="', '').replace('">', ' ').replace('</a>', '') for x in y] for y in data]
    return data

def extract_data(data, year):
    table = []
    for row in data[1:]:
        try:
            company = row[0][row[0].find(str(year)) + 5:]
            job = row[1][row[1].find(str(year)) + 5:]
            salary = row[2][row[2].find("_blank") + 7:]
            location = row[3][row[3].find(str(year)) + 5:]
            year = year
            if any(x in job.lower() for x in senior_roles) or any(x in job for x in seniority):
                continue
            table.append([company, job, salary, location, year])
        except Exception as e:
            print(row)
            continue

    return table

def table_to_dataframe(data):
    df = pd.DataFrame(data, columns=['Company', 'Job', 'Salary', 'Location', 'Year'])
    return df

In [95]:
jobx = 'data+scientist'
year = 2023
data = get_data(jobx, year)
data = process_data(data)
data = extract_data(data, year)
df = table_to_dataframe(data)
print(df)

[]
[]
[]
[]
[]
                   Company             Job   Salary        Location  Year
0       ISTEAM ACADEMY INC  DATA SCIENTIST   56,000  BURLINGTON, MA  2023
1     UNIHOME NEW YORK INC  DATA SCIENTIST   57,440        TROY, MI  2023
2         LINEAR STATS INC  DATA SCIENTIST   58,000   CHAMPAIGN, IL  2023
3         LINEAR STATS INC  DATA SCIENTIST   60,000   CHAMPAIGN, IL  2023
4         LINEAR STATS INC  DATA SCIENTIST   60,000   CHAMPAIGN, IL  2023
...                    ...             ...      ...             ...   ...
2214         BYTEDANCE INC  DATA SCIENTIST  315,000    SAN JOSE, CA  2023
2215               USG INC  DATA SCIENTIST  910,000      DULUTH, GA  2023
2216               USG INC  DATA SCIENTIST  910,000      DULUTH, GA  2023
2217               USG INC  DATA SCIENTIST  910,000      DULUTH, GA  2023
2218               USG INC  DATA SCIENTIST  910,000      DULUTH, GA  2023

[2219 rows x 5 columns]


In [96]:
with open('jobs_to_scrape.txt', 'r') as f:
    jobs = f.readlines()
    jobs = [x.strip() for x in jobs]

jobs_to_scrape = []
for row in jobs:
    data = row.split(':')
    name = data[0]
    print(name)
    data = data[1].split(',')
    exclude = [x[1:] for x in data if x[0] == '-']
    include = [x for x in data if x[0] != '-']
    jobs_to_scrape.append([name, include, exclude])

Machine Learning
Artifical Intelligence
Data Science
Data Analyst
Data Engineer
Graphics
AR VR
Deep Learning
Cybersecurity Engineer
Security Engineer
Embedded Systems Engineer
Full-Stack Developer
Front-end Developer
DevOps Engineer
Cloud Architect
Software Architect
Data Architect
Project Manager
QA Engineer
Mobile Developer


In [97]:
jobs_to_scrape

[['Machine Learning', ['machine learnin', 'ml'], []],
 ['Artifical Intelligence',
  ['artificial', 'ai'],
  ['AIR', 'AIRCRAFT', 'AIRLINE', 'AIX', 'AIR QUALITY', 'AIRPORT']],
 ['Data Science', ['data scien'], []],
 ['Data Analyst', ['data ana'], []],
 ['Data Engineer', ['data eng'], []],
 ['Graphics',
  ['gpu', 'graphics software engineer', 'graphics hardware engineer'],
  []],
 ['AR VR',
  ['ar ', 'vr ', 'virtual reality', 'augmented'],
  ['architect',
   'arc',
   'art',
   'ara',
   'area',
   'archer',
   'ariba',
   'archivist',
   'ARCHITECH',
   'ARABIC',
   'ARCHEOLOGIIST']],
 ['Deep Learning', ['deep learnin'], []],
 ['Cybersecurity Engineer', ['cyber'], []],
 ['Security Engineer', ['security'], []],
 ['Embedded Systems Engineer', ['embedded'], []],
 ['Full-Stack Developer', ['full stac', 'web dev'], []],
 ['Front-end Developer', ['front'], []],
 ['DevOps Engineer', ['devop'], []],
 ['Cloud Architect', ['cloud arc'], []],
 ['Software Architect', ['software arc'], []],
 ['Data A

In [98]:
for i in range(2012, 2024):
    YEAR = i
    os.makedirs(f'data/{i}', exist_ok=True)
    for name, jobs, exclude in jobs_to_scrape:
        dfs = []
        for job in jobs:
            print(f'scraping {job}')
            data = get_data(job, YEAR)
            data = process_data(data)
            data = extract_data(data, YEAR)
            df = table_to_dataframe(data)
            for word in exclude:
                df = df[~df['Job'].str.contains(str.upper(word))]
            df['Salary'] = df['Salary'].str.replace(',', '').str.replace('$', '').astype(float)
            dfs.append(df)
        df = pd.concat(dfs)
        df.to_csv(f'data/{i}/{name}.csv', index=False)
        print(f'{name} done')
        print('-------------------')
    print(f'{YEAR} done')

scraping machine learnin
scraping ml
Machine Learning done
-------------------
scraping artificial
scraping ai
Artifical Intelligence done
-------------------
scraping data scien
Data Science done
-------------------
scraping data ana
Data Analyst done
-------------------
scraping data eng
Data Engineer done
-------------------
scraping gpu
scraping graphics software engineer
scraping graphics hardware engineer
Graphics done
-------------------
scraping ar 
scraping vr 
scraping virtual reality
scraping augmented
AR VR done
-------------------
scraping deep learnin
Deep Learning done
-------------------
scraping cyber
Cybersecurity Engineer done
-------------------
scraping security
Security Engineer done
-------------------
scraping embedded
Embedded Systems Engineer done
-------------------
scraping full stac
scraping web dev
Full-Stack Developer done
-------------------
scraping front
Front-end Developer done
-------------------
scraping devop
DevOps Engineer done
------------------

In [99]:
# Summary Page
for year in os.listdir('data'):
    records = 0
    for file in os.listdir(f'data/{year}'):
        df = pd.read_csv(f'data/{year}/{file}')
        print(f'{file}: {df.shape[0]} records')
        records += df.shape[0]
    print(f'{year}: {records} records')
    print('-------------------')

AR VR.csv: 0 records
Artifical Intelligence.csv: 0 records
Cloud Architect.csv: 0 records
Cybersecurity Engineer.csv: 0 records
Data Analyst.csv: 0 records
Data Architect.csv: 0 records
Data Engineer.csv: 0 records
Data Science.csv: 0 records
Deep Learning.csv: 0 records
DevOps Engineer.csv: 0 records
Embedded Systems Engineer.csv: 0 records
Front-end Developer.csv: 0 records
Full-Stack Developer.csv: 0 records
Graphics.csv: 0 records
Machine Learning.csv: 0 records
Mobile Developer.csv: 0 records
Project Manager.csv: 0 records
QA Engineer.csv: 0 records
Security Engineer.csv: 0 records
Software Architect.csv: 1 records
2012: 1 records
-------------------
AR VR.csv: 0 records
Artifical Intelligence.csv: 0 records
Cloud Architect.csv: 0 records
Cybersecurity Engineer.csv: 0 records
Data Analyst.csv: 1 records
Data Architect.csv: 2 records
Data Engineer.csv: 3 records
Data Science.csv: 1 records
Deep Learning.csv: 0 records
DevOps Engineer.csv: 0 records
Embedded Systems Engineer.csv: 0 